En este proyecto, hemos desarrollado una serie de consultas SQL aplicadas a la base de datos de una cadena de supermercados estadounidense, con el propósito de analizar y comprender aspectos clave del negocio. El análisis abarca áreas como ventas, segmentación y comportamiento del cliente, rendimiento geográfico y distribución.

Estas consultas proporcionan una visión completa del funcionamiento y desempeño de la cadena de supermercados, sirviendo como herramienta para la toma de decisiones estratégicas y operativas.

In [3]:
import pandas as pd
import duckdb

In [4]:
sales=pd.read_csv('/content/Sales.csv')
products=pd.read_csv('/content/Products.csv')
customers=pd.read_csv('/content/Customers.csv')

In [5]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Row ID       9994 non-null   int64  
 1   Order ID     9994 non-null   object 
 2   Order Date   9994 non-null   object 
 3   Ship Date    9994 non-null   object 
 4   Ship Mode    9994 non-null   object 
 5   Customer ID  9994 non-null   object 
 6   Product ID   9994 non-null   object 
 7   Sales        9994 non-null   float64
 8   Quantity     9994 non-null   int64  
 9   Discount     9994 non-null   float64
 10  Profit       9994 non-null   float64
dtypes: float64(3), int64(2), object(6)
memory usage: 859.0+ KB


In [6]:
sales = sales.rename(columns={'Row ID': 'Row_ID',
                              'Order ID': 'Order_ID',
                              'Order Date': 'Order_Date',
                              'Ship Date': 'Ship_Date',
                              'Ship Mode': 'Ship_Mode',
                              'Customer ID': 'Customer_ID',
                              'Product ID': 'Product_ID'})

In [7]:
sales['Order_Date'] = pd.to_datetime(sales['Order_Date'])
sales['Ship_Date'] = pd.to_datetime(sales['Ship_Date'])

In [8]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1862 entries, 0 to 1861
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product ID    1862 non-null   object
 1   Category      1862 non-null   object
 2   Sub-Category  1862 non-null   object
 3   Product Name  1862 non-null   object
dtypes: object(4)
memory usage: 58.3+ KB


In [9]:
products = products.rename(columns={'Product ID': 'Product_ID',
                                    'Sub-Category': 'Sub_Category',
                                    'Product Name': 'Product_Name'})

In [10]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Customer ID    793 non-null    object
 1   Customer Name  793 non-null    object
 2   Segment        793 non-null    object
 3   City           793 non-null    object
 4   State          793 non-null    object
 5   Region         793 non-null    object
 6   Postal Code    793 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 43.5+ KB


In [11]:
customers = customers.rename(columns={'Customer ID': 'Customer_ID',
                                    'Customer Name': 'Customer_Name',
                                    'Postal Code': 'Postal_Code'})

In [12]:
#Consulta de productos más vendidos

query1 = """
SELECT
    p.Product_Name,
    SUM(s.Quantity) AS Productos_Vendidos
FROM sales s
JOIN products p ON s.Product_ID = p.Product_ID
GROUP BY p.Product_Name
ORDER BY Productos_Vendidos DESC
LIMIT 10;
"""
result = duckdb.query(query1).df()
print(result)

                                        Product_Name  Productos_Vendidos
0                                            Staples               215.0
1                                    Staple envelope               170.0
2                                  Easy-staple paper               150.0
3                            Staples in misc. colors                86.0
4                 Logitech P710e Mobile Speakerphone                75.0
5                         KI Adjustable-Height Table                74.0
6                            Storex Dura Pro Binders                71.0
7                            Avery Non-Stick Binders                71.0
8                                         Xerox 1881                70.0
9  GBC Premium Transparent Covers with Diagonal L...                67.0


In [13]:
#Consulta de productos con mayor ingreso

query2 = """
SELECT p.Product_Name, SUM(s.Sales) AS Total_Ventas
FROM sales s
JOIN products p ON s.Product_ID = p.Product_ID
GROUP BY p.Product_Name
ORDER BY Total_Ventas DESC
LIMIT 10;
"""
result2 = duckdb.query(query2).df()
print(result2)

                                        Product_Name  Total_Ventas
0              Canon imageCLASS 2200 Advanced Copier     61599.824
1  Fellowes PB500 Electric Punch Plastic Comb Bin...     27453.384
2  Cisco TelePresence System EX90 Videoconferenci...     22638.480
3       HON 5400 Series Task Chairs for Big and Tall     21870.576
4         GBC DocuBind TL300 Electric Binding System     19823.479
5   GBC Ibimaster 500 Manual ProClick Binding System     19024.500
6               Hewlett Packard LaserJet 3310 Copier     18839.686
7  HP Designjet T520 Inkjet Large Format Printer ...     18374.895
8          GBC DocuBind P400 Electric Binding System     17965.068
9        High Speed Automatic Electric Letter Opener     17030.312


In [14]:
#Consulta de mayores compradores agrupados por ciudad y tipo de cliente

query3 = """
SELECT
    c.City,
    c.Segment,
    SUM(s.Sales) AS Total_Ventas
FROM sales s
JOIN customers c ON s.Customer_ID = c.Customer_ID
GROUP BY c.City, c.Segment
ORDER BY Total_Ventas DESC
LIMIT 10;
"""
result3 = duckdb.query(query3).df()
print(result3)

            City      Segment  Total_Ventas
0  New York City     Consumer    78401.7091
1  New York City    Corporate    75613.5246
2    Los Angeles     Consumer    69250.1000
3        Seattle     Consumer    68762.2098
4  San Francisco     Consumer    62007.0800
5   Philadelphia     Consumer    61693.3002
6   Philadelphia    Corporate    60299.2408
7  New York City  Home Office    56911.7584
8        Houston     Consumer    45581.9396
9  San Francisco    Corporate    40817.7835


In [15]:
#Consulta de Estados con mas ventas

query4 = """
SELECT
    c.State,
    SUM(s.Sales) AS Total_Ventas
FROM sales s
JOIN customers c ON s.Customer_ID = c.Customer_ID
GROUP BY c.State
ORDER BY Total_Ventas DESC
LIMIT 10;
"""
result4 = duckdb.query(query4).df()
print(result4)

            State  Total_Ventas
0      California   451036.5823
1        New York   279549.8235
2           Texas   192758.2049
3    Pennsylvania   142838.5510
4      Washington   133177.2518
5  North Carolina   116635.4615
6        Illinois   112819.7720
7         Arizona    81986.1230
8            Ohio    74771.3300
9        Michigan    58076.8598


In [16]:
#Consulta de ticket promedio en cada Estado

query5 = """
SELECT
    c.State,
    SUM(s.Sales)/COUNT(DISTINCT c.Customer_ID) AS Tiket_Promedio
FROM sales s
JOIN customers c ON s.Customer_ID = c.Customer_ID
GROUP BY c.State
ORDER BY Tiket_Promedio DESC;
"""
result5 = duckdb.query(query5).df()
print(result5)

                   State  Tiket_Promedio
0                 Oregon     5775.723667
1          New Hampshire     4860.073000
2              Wisconsin     4552.250111
3                   Iowa     4142.049000
4                Arizona     3904.101095
5         North Carolina     3887.848717
6            Mississippi     3869.449000
7              Minnesota     3850.952346
8               Michigan     3629.803737
9               Delaware     3509.347500
10            Washington     3504.664521
11               Alabama     3448.776867
12              Nebraska     3246.205000
13              New York     3213.216362
14              Virginia     3101.419867
15               Georgia     3023.537941
16              Colorado     2876.155810
17          Pennsylvania     2856.771020
18         Massachusetts     2841.134500
19             Tennessee     2825.660246
20            California     2801.469455
21             Louisiana     2770.978000
22              Illinois     2751.701756
23              

In [17]:
#Consulta de ventas por categoría de producto en cada región.

query6 = """
SELECT
    c.Region,
    p.Category,
    SUM(s.Sales) AS Total_Ventas
FROM sales s
JOIN customers c ON s.Customer_ID = c.Customer_ID
JOIN products p ON s.Product_ID = p.Product_ID
GROUP BY c.Region, p.Category
ORDER BY c.Region, Total_Ventas DESC;
"""
result6 = duckdb.query(query6).df()
print(result6)

     Region         Category  Total_Ventas
0   Central       Technology   184752.7770
1   Central        Furniture   176419.4072
2   Central  Office Supplies   157627.9480
3      East       Technology   246022.2590
4      East  Office Supplies   193251.3320
5      East        Furniture   172460.7085
6     South       Technology   160094.5890
7     South  Office Supplies   128092.6880
8     South        Furniture   113844.7063
9      West        Furniture   279274.9733
10     West       Technology   245284.4080
11     West  Office Supplies   240075.0640


In [18]:
#Consulta de los 10 productos con mayor rentabilidad

query7 = """
SELECT
    p.Product_Name,
    SUM(s.Profit) AS Rentabilidad_Total
FROM sales s
JOIN products p ON s.Product_ID = p.Product_ID
GROUP BY p.Product_Name
ORDER BY Rentabilidad_Total DESC
LIMIT 10;
"""
result7 = duckdb.query(query7).df()
print(result7)

                                        Product_Name  Rentabilidad_Total
0              Canon imageCLASS 2200 Advanced Copier          25199.9280
1  Fellowes PB500 Electric Punch Plastic Comb Bin...           7753.0390
2               Hewlett Packard LaserJet 3310 Copier           6983.8836
3                 Canon PC1060 Personal Laser Copier           4570.9347
4          Logitech G19 Programmable Gaming Keyboard           4425.3432
5  HP Designjet T520 Inkjet Large Format Printer ...           4094.9766
6                  Ativa V4110MDD Micro-Cut Shredder           3772.9461
7   3D Systems Cube Printer, 2nd Generation, Magenta           3717.9714
8               Ibico EPK-21 Electric Binding System           3345.2823
9                  Zebra ZM400 Thermal Label Printer           3343.5360


In [19]:
#Consulta de los 10 productos con menor rentabilidad

query8 = """
SELECT
    p.Product_Name,
    SUM(s.Profit) AS Rentabilidad_Total
FROM sales s
JOIN products p ON s.Product_ID = p.Product_ID
GROUP BY p.Product_Name
ORDER BY Rentabilidad_Total ASC
LIMIT 10;
"""
result8 = duckdb.query(query8).df()
print(result8)

                                        Product_Name  Rentabilidad_Total
0          Cubify CubeX 3D Printer Double Head Print          -8879.9704
1          Lexmark MX611dhe Monochrome Laser Printer          -4589.9730
2          Cubify CubeX 3D Printer Triple Head Print          -3839.9904
3  Chromcraft Bull-Nose Wood Oval Conference Tabl...          -2876.1156
4  Bush Advantage Collection Racetrack Conference...          -1934.3976
5          GBC DocuBind P400 Electric Binding System          -1878.1662
6  Cisco TelePresence System EX90 Videoconferenci...          -1811.0784
7  Martin Yale Chadless Opener Electric Letter Op...          -1299.1836
8                       Balt Solid Wood Round Tables          -1201.0581
9  BoxOffice By Design Rectangular and Half-Moon ...          -1148.4375


In [20]:
#Consulta de rentabilidad promedio en base al descuento aplicado

query9 = """
SELECT
    CASE
        WHEN Discount = 0 THEN '0%'
        WHEN Discount BETWEEN 0.01 AND 0.1 THEN '1-10%'
        WHEN Discount BETWEEN 0.11 AND 0.2 THEN '11-20%'
        WHEN Discount BETWEEN 0.21 AND 0.3 THEN '21-30%'
        WHEN Discount BETWEEN 0.31 AND 0.4 THEN '31-40%'
        WHEN Discount > 0.4 THEN 'Más de 40%'
    END AS Rango_Descuento,
    COUNT(*) AS Cantidad_Ventas,
    AVG(Profit) AS Rentabilidad_Promedio,

FROM sales
GROUP BY Rango_Descuento
ORDER BY Rango_Descuento;
"""
result9 = duckdb.query(query9).df()
print(result9)

  Rango_Descuento  Cantidad_Ventas  Rentabilidad_Promedio
0              0%             4798              66.900292
1           1-10%               94              96.055074
2          11-20%             3709              24.738824
3          21-30%              227             -45.679636
4          31-40%              233            -109.219691
5      Más de 40%              933            -106.708028


In [21]:
#Consulta de rentabilidad total por Estado

query10 = """
SELECT
    c.State,
    SUM(s.Profit) AS Rentabilidad_Total
FROM sales s
JOIN customers c ON s.Customer_ID = c.Customer_ID
GROUP BY c.State
ORDER BY Rentabilidad_Total DESC;
"""
result10 = duckdb.query(query10).df()
print(result10)

                   State  Rentabilidad_Total
0             California          59398.3125
1               New York          58177.8341
2             Washington          24405.7966
3                  Texas          20528.9110
4           Pennsylvania          13604.9350
5                Georgia          12781.3426
6                Arizona           9563.2001
7               Illinois           9560.1456
8              Wisconsin           8569.8697
9               Michigan           7752.2969
10             Minnesota           7202.5225
11              Virginia           6940.1112
12                  Ohio           5985.8870
13         Massachusetts           5905.5446
14              Kentucky           4513.3140
15             Tennessee           3434.2765
16              Delaware           3336.3827
17               Alabama           2845.0624
18               Indiana           2707.3495
19             Louisiana           2659.2401
20            New Jersey           2336.7532
21        

In [22]:
#Consulta de ventas totales durante cada mes entre 2014 y 2017

query11 = """
SELECT
    EXTRACT(MONTH FROM Order_Date) AS Mes,
    EXTRACT(YEAR FROM Order_Date) AS Año,
    SUM(Sales) AS Total_Ventas
FROM sales
GROUP BY
    EXTRACT(MONTH FROM Order_Date),
    EXTRACT(YEAR FROM Order_Date)
ORDER BY Año, Mes;
"""
result11 = duckdb.query(query11).df()
print(result11)


    Mes   Año  Total_Ventas
0     1  2014    14236.8950
1     2  2014     4519.8920
2     3  2014    55691.0090
3     4  2014    28295.3450
4     5  2014    23648.2870
5     6  2014    34595.1276
6     7  2014    33946.3930
7     8  2014    27909.4685
8     9  2014    81777.3508
9    10  2014    31453.3930
10   11  2014    78628.7167
11   12  2014    69545.6205
12    1  2015    18174.0756
13    2  2015    11951.4110
14    3  2015    38726.2520
15    4  2015    34195.2085
16    5  2015    30131.6865
17    6  2015    24797.2920
18    7  2015    28765.3250
19    8  2015    36898.3322
20    9  2015    64595.9180
21   10  2015    31404.9235
22   11  2015    75972.5635
23   12  2015    74919.5212
24    1  2016    18542.4910
25    2  2016    22978.8150
26    3  2016    51715.8750
27    4  2016    38750.0390
28    5  2016    56987.7280
29    6  2016    40344.5340
30    7  2016    39261.9630
31    8  2016    31115.3743
32    9  2016    73410.0249
33   10  2016    59687.7450
34   11  2016    794

In [23]:
#Consulta de venta promedio por dia de la semana

query12 = """
SELECT
    CASE DAYOFWEEK(Order_Date)
        WHEN 0 THEN 'Domingo'
        WHEN 1 THEN 'Lunes'
        WHEN 2 THEN 'Martes'
        WHEN 3 THEN 'Miércoles'
        WHEN 4 THEN 'Jueves'
        WHEN 5 THEN 'Viernes'
        WHEN 6 THEN 'Sábado'
    END AS dia_semana,
    AVG(Sales) AS venta_promedio
FROM sales
GROUP BY DAYOFWEEK(Order_Date)
ORDER BY DAYOFWEEK(Order_Date)
"""
result12 = duckdb.query(query12).df()
print(result12)

  dia_semana  venta_promedio
0    Domingo      225.353127
1      Lunes      229.255914
2     Martes      259.683094
3  Miércoles      237.350337
4     Jueves      220.072302
5    Viernes      235.254268
6     Sábado      216.304944


In [24]:
#Consulta de retraso promedio en base a la región y la modalidad de envío

query13 = """
SELECT
    c.Region,
    s.Ship_Mode,
    AVG(DATEDIFF('day', s.Order_Date, s.Ship_Date)) AS Retraso_Promedio_Días
FROM sales s
JOIN customers c ON s.Customer_ID = c.Customer_ID
GROUP BY
    c.Region,
    s.Ship_Mode
ORDER BY
    Retraso_Promedio_Días DESC;

"""
result13 = duckdb.query(query13).df()
print(result13)

     Region       Ship_Mode  Retraso_Promedio_Días
0      West  Standard Class               5.076642
1   Central  Standard Class               5.011080
2     South  Standard Class               4.953557
3      East  Standard Class               4.951694
4      East    Second Class               3.304505
5     South    Second Class               3.247525
6      West    Second Class               3.204437
7   Central    Second Class               3.197368
8     South     First Class               2.283898
9      West     First Class               2.185115
10     East     First Class               2.179825
11  Central     First Class               2.108696
12  Central        Same Day               0.067114
13     West        Same Day               0.055556
14     East        Same Day               0.025478
15    South        Same Day               0.013333
